In [1]:
# %load /home/sjkim/.jupyter/head.py
# %%writefile /home/sjkim/.jupyter/head.py
# %load /home/sjkim/.jupyter/head.py
%matplotlib inline
%load_ext autoreload 
%autoreload 2
from importlib import reload

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import os

### Lambda

In [2]:
from keras.layers import Lambda, Input
from keras.models import Model

x = Input((1,))
y = Lambda(lambda x: x + 1)(x)
m = Model(x, y)

yp = m.predict_on_batch([1, 2, 3])
print("np.array([1,2,3]) + 1:")
print(yp)

Using TensorFlow backend.


np.array([1,2,3]) + 1:
[[2.]
 [3.]
 [4.]]


### Lambda functions

In [3]:
from keras.layers import Lambda, Input
from keras.models import Model

def kproc(x):
    return x ** 2 + 2 * x + 1

def kshape(input_shape):
    return input_shape

x = Input((1,))
y = Lambda(kproc, kshape)(x)
m = Model(x, y)

yp = m.predict_on_batch([1, 2, 3])
print("x**2 + 2 * x + 1: x = np.array([1,2,3]):")
print(yp)

x**2 + 2 * x + 1: x = np.array([1,2,3]):
[[ 4.]
 [ 9.]
 [16.]]


### engine.topology.Layer

In [4]:
from keras.layers import Lambda, Input
from keras.models import Model
from keras.engine.topology import Layer

class Mult(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def call(self, x):
        return x ** 2 + 2 * x + 1

x = Input((1,))
y = Mult()(x)
m = Model(x, y)

yp = m.predict_on_batch([1, 2, 3])
print("x**2 + 2 * x + 1: x = np.array([1,2,3]):")
print(yp)

x**2 + 2 * x + 1: x = np.array([1,2,3]):
[[ 4.]
 [ 9.]
 [16.]]


### Backend for Lambda

In [5]:
from keras.layers import Lambda, Input
from keras.models import Model
from keras import backend as K

def kproc_concat(x):    
    m = K.mean(x, axis=1, keepdims=True)
    d1 = K.abs(x - m)
    d2 = K.square(x - m)
    return K.concatenate([x, d1, d2], axis=1)

def kshape_concat(input_shape):
    output_shape = list(input_shape)
    output_shape[1] *= 3
    return tuple(output_shape)

x = Input((3,))
y = Lambda(kproc_concat, kshape_concat)(x)
m = Model(x, y)

yp = m.predict_on_batch([[1, 2, 3], [3, 4, 8]])
print(yp)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 2 arrays: [array([[1],
       [2],
       [3]]), array([[3],
       [4],
       [8]])]...

### TF engine for Lambda

In [6]:
from keras.layers import Lambda, Input
from keras.models import Model
import tensorflow as tf

def kproc_concat(x):    
    m = tf.reduce_mean(x, axis=1, keep_dims=True)
    d1 = tf.abs(x - m)
    d2 = tf.square(x - m)
    return tf.concat([x, d1, d2], axis=1)

def kshape_concat(input_shape):
    output_shape = list(input_shape)
    output_shape[1] *= 3
    return tuple(output_shape)

x = Input((3,))
y = Lambda(kproc_concat, kshape_concat)(x)
m = Model(x, y)

yp = m.predict_on_batch([[1, 2, 3], [3, 4, 8]])
print(yp)

TypeError: reduce_mean() got an unexpected keyword argument 'keep_dims'